Plots for super-greenhouse climatology \
Venv sge_env

Maura Dewey (maura.dewey@misu.su.se), 2023

### 2. paper figures
from data prep notebook or repository load the saved timeseries file

In [ ]:
import netCDF4 as nc
import xarray as xr
from pyhdf.SD import SD, SDC
import numpy as np
import pandas as pd
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import datetime, timedelta
from os import path
import matplotlib.gridspec as gridspec
import seaborn as sns 
import cartopy
from statsmodels.tsa.seasonal import STL
import cmasher as cm


In [ ]:
SGE_ts = xr.open_dataset('SGE_timeseries.nc')

In [ ]:
#clean data
#linearly interpolate NaNs for missing days, since we need monotonic data for ffts, etc.
# days with values well outside expected daily change (due to lack of satellite coverage for example are removed and then also linearly interpolated)

frc_all = SGE_ts.SGE_frac_all*100
frc_all = frc_all.where(abs(frc_all.diff('time')) < 5).where(frc_all>13)
frc_all_clean = frc_all.interpolate_na(dim='time',method='linear')
frc_all_clean = frc_all_clean.dropna(dim='time') #drop missing days from ends of timeseries

frc_clr = SGE_ts.SGE_frac_clr*100
frc_clr = frc_clr.where(frc_clr>0)
frc_clr_clean = frc_clr.interpolate_na(dim='time',method='linear')
frc_clr_clean = frc_clr_clean.dropna(dim='time')

pw_all = SGE_ts.SGE_pw_all/1e15
pw_all = pw_all.where(pw_all>1.75)
pw_all_clean = pw_all.interpolate_na(dim='time',method='linear')
pw_all_clean = pw_all_clean.dropna(dim='time')

pw_clr = SGE_ts.SGE_pw_clr/1e12
pw_clr = pw_clr.where(pw_clr>0)
pw_clr_clean = pw_clr.interpolate_na(dim='time',method='linear')
pw_clr_clean = pw_clr_clean.dropna(dim='time')

In [ ]:
#Figure 1: timeseries and deseasonalized trends of area fraction and power for all-sky and clear-sky data
# Trends computed with STL (seasonal trend-decomposition with LOESS, periodicity=365 days and robust to outliers)

#STL https://www.statsmodels.org/dev/examples/notebooks/generated/stl_decomposition.html#
#R. B. Cleveland, W. S. Cleveland, J.E. McRae, and I. Terpenning (1990) STL: A Seasonal-Trend Decomposition Procedure Based on LOESS. Journal of Official Statistics, 6, 3-73.

stl_frALL = STL(frc_all_clean, period=365, seasonal=7, robust=True)
res_frALL = stl_frALL.fit()

stl_frCLR = STL(frc_clr_clean, period=365, seasonal=7, robust=True)
res_frCLR = stl_frCLR.fit()

stl_pwALL = STL(pw_all_clean, period=365, seasonal=7, robust=True)
res_pwALL = stl_pwALL.fit()

stl_pwCLR = STL(pw_clr_clean, period=365, seasonal=7, robust=True)
res_pwCLR = stl_pwCLR.fit()

In [ ]:
#testing STL parameters:

stl_s13 = STL(frc_all_clean, period=365, seasonal=13, robust=True)
res_s13 = stl_s13.fit()

stl_s7 = STL(frc_all_clean, period=365, seasonal=7, robust=True)
res_s7 = stl_s7.fit()

stl_p183 = STL(frc_all_clean, period=183, seasonal=7, robust=True)
res_p183 = stl_p183.fit()

stl_s25 = STL(frc_all_clean, period=365, seasonal=25, robust=True)
res_s25 = stl_s25.fit()

plt.plot(res_s7.trend, label='seasonal=7')
plt.plot(res_s13.trend,label='seasonal=13')
plt.plot(res_p183.trend, label='period=183')
plt.plot(res_s25.trend, label='seasonal=25')
plt.legend()

In [ ]:
#Figure 1, timeseries trends
with sns.plotting_context("talk"):

    fig, axes = plt.subplots(2,2, figsize=(12,8), sharex=True)
    axes[0,0].plot(frc_all_clean.time, frc_all_clean,color='orange',linewidth=1,alpha=0.5)
    axes[0,0].plot(frc_all_clean.time, res_frALL.trend,color='sienna')
    axes[0,0].autoscale(enable=True, axis='x', tight=True)
    axes[0,0].set_title('All-sky')
    axes[0,0].set_ylabel('daily SGE fraction [%]')
    
    axes[0,1].plot(frc_clr_clean.time, frc_clr_clean, linewidth=1,color='c')
    axes[0,1].plot(frc_clr_clean.time, res_frCLR.trend, color='b')
    axes[0,1].set_title('Clear-sky')
    axes[0,1].autoscale(enable=True, axis='x', tight=True)
    #axes[0,1].yaxis.tick_right()
    axes[0,1].yaxis.set_label_position("right")
    axes[0,1].set_ylabel('daily SGE fraction [%]')

    axes[1,0].plot(pw_all_clean.time, pw_all_clean,color='orange',linewidth=1,alpha=0.5)
    axes[1,0].plot(pw_all_clean.time, res_pwALL.trend,color='sienna')
    axes[1,0].autoscale(enable=True, axis='x', tight=True)
    axes[1,0].set_ylabel('daily SGE strength [TW]')
    axes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), rotation = 45)

    axes[1,1].plot(pw_clr_clean.time, pw_clr_clean, linewidth=1,color='c')
    axes[1,1].plot(pw_clr_clean.time, res_pwCLR.trend, color='b')
    axes[1,1].autoscale(enable=True, axis='x', tight=True)
    #axes[1,1].yaxis.tick_right()
    axes[1,1].yaxis.set_label_position("right")
    axes[1,1].set_ylabel('daily SGE strength [PW]')
    axes[1,1].set_xticklabels(axes[1,1].get_xticklabels(), rotation = 45)

fig.tight_layout()

In [ ]:
#plot full decomposition of each timeseries for supplimentary:

with sns.plotting_context("talk"):
    fig, axes = plt.subplots(4,1, figsize=(8,8), sharex=True)
    axes[0].plot(frc_all_clean.time, frc_all_clean,color='orange')
    axes[0].autoscale(enable=True, axis='x', tight=True)
    axes[0].set_title('All-sky SGE fraction')
    axes[0].set_ylabel('timeseries [%]')

    axes[1].plot(frc_all_clean.time, res_frALL.trend,color='gold')
    axes[1].autoscale(enable=True, axis='x', tight=True)
    axes[1].set_ylabel('trend')

    axes[2].plot(frc_all_clean.time, res_frALL.seasonal,color='gold')
    axes[2].autoscale(enable=True, axis='x', tight=True)
    axes[2].set_ylabel('seasonal')

    axes[3].plot(frc_all_clean.time, res_frALL.resid,color='gold')
    axes[3].autoscale(enable=True, axis='x', tight=True)
    axes[3].set_ylabel('residual')

fig.tight_layout()

In [ ]:
#plot full decomposition of each timeseries for supplimentary:

with sns.plotting_context("talk"):
    fig, axes = plt.subplots(4,1, figsize=(8,8), sharex=True)
    axes[0].plot(pw_all_clean.time, pw_all_clean,color='orange')
    axes[0].autoscale(enable=True, axis='x', tight=True)
    axes[0].set_title('All-sky SGE strength')
    axes[0].set_ylabel('timeseries [TW]')

    axes[1].plot(pw_all_clean.time, res_pwALL.trend,color='gold')
    axes[1].autoscale(enable=True, axis='x', tight=True)
    axes[1].set_ylabel('trend')

    axes[2].plot(pw_all_clean.time, res_pwALL.seasonal,color='gold')
    axes[2].autoscale(enable=True, axis='x', tight=True)
    axes[2].set_ylabel('seasonal')

    axes[3].plot(pw_all_clean.time, res_pwALL.resid,color='gold')
    axes[3].autoscale(enable=True, axis='x', tight=True)
    axes[3].set_ylabel('residual')

fig.tight_layout()

In [ ]:
#plot full decomposition of each timeseries for supplimentary:

with sns.plotting_context("talk"):
    fig, axes = plt.subplots(4,1, figsize=(8,8), sharex=True)
    axes[0].plot(frc_clr_clean.time, frc_clr_clean,color='mediumblue')
    axes[0].autoscale(enable=True, axis='x', tight=True)
    axes[0].set_title('Clear-sky SGE fraction')
    axes[0].set_ylabel('timeseries [%]')

    axes[1].plot(frc_clr_clean.time, res_frCLR.trend,color='skyblue')
    axes[1].autoscale(enable=True, axis='x', tight=True)
    axes[1].set_ylabel('trend')

    axes[2].plot(frc_clr_clean.time, res_frCLR.seasonal,color='skyblue')
    axes[2].autoscale(enable=True, axis='x', tight=True)
    axes[2].set_ylabel('seasonal')

    axes[3].plot(frc_clr_clean.time, res_frCLR.resid,color='skyblue')
    axes[3].autoscale(enable=True, axis='x', tight=True)
    axes[3].set_ylabel('residual')

fig.tight_layout()

In [ ]:
#plot full decomposition of each timeseries for supplimentary:

with sns.plotting_context("talk"):
    fig, axes = plt.subplots(4,1, figsize=(8,8), sharex=True)
    axes[0].plot(pw_clr_clean.time, pw_clr_clean,color='mediumblue')
    axes[0].autoscale(enable=True, axis='x', tight=True)
    axes[0].set_title('Clear-sky SGE strength')
    axes[0].set_ylabel('timeseries [PW]')

    axes[1].plot(pw_clr_clean.time, res_pwCLR.trend,color='skyblue')
    axes[1].autoscale(enable=True, axis='x', tight=True)
    axes[1].set_ylabel('trend')

    axes[2].plot(pw_clr_clean.time, res_pwCLR.seasonal,color='skyblue')
    axes[2].autoscale(enable=True, axis='x', tight=True)
    axes[2].set_ylabel('seasonal')

    axes[3].plot(pw_clr_clean.time, res_pwCLR.resid,color='skyblue')
    axes[3].autoscale(enable=True, axis='x', tight=True)
    axes[3].set_ylabel('residual')

fig.tight_layout()

In [ ]:
#Figure 2: Annual cycles

#transform to dataframe with column for each year
#df_frALL = frc_all_clean.resample(time='MS').mean().to_dataframe()
#df_frALL['year'] = df_frALL.index.year
#df_frALL['month'] = df_frALL.index.month

df_pwALL = pw_all.to_dataframe()
df_pwALL['year'] = df_pwALL.index.year
df_pwALL['DayOfYear'] = df_pwALL.index.dayofyear
df_frALL = frc_all.to_dataframe()
df_ALL = pd.concat([df_pwALL, df_frALL], axis=1)

df_pwCLR = pw_clr.to_dataframe()
df_pwCLR['year'] = df_pwCLR.index.year
df_pwCLR['DayOfYear'] = df_pwCLR.index.dayofyear
df_frCLR = frc_clr.to_dataframe()
df_CLR = pd.concat([df_pwCLR, df_frCLR], axis=1)

In [ ]:

with sns.plotting_context("talk"):

    fig = plt.figure(figsize=(14,14))
    gs_left = gridspec.GridSpec(ncols=4, nrows=4, hspace = 0.0, wspace = 0.5)
    gs_right = gridspec.GridSpec(ncols=4, nrows=4, hspace=0.6, wspace=1)

    ax1 = fig.add_subplot(gs_left[0,0:2])
    sns.lineplot(ax=ax1,data=df_ALL, x='DayOfYear',y='SGE_frac_all',hue='year',palette='copper',legend=False)
    ax1.set_ylabel('fraction [%]')

    ax2 = fig.add_subplot(gs_left[1,0:2])
    sns.lineplot(ax=ax2,data=df_ALL, x='DayOfYear',y='SGE_pw_all',hue='year',palette='copper',legend=False)
    ax2.set_ylabel('strength [TW]')

    ax3 = fig.add_subplot(gs_left[2,0:2])
    sns.lineplot(data=df_CLR, x='DayOfYear',y='SGE_frac_clr',hue='year',palette=cm.get_sub_cmap('bone', 0.0, 0.7),legend=False)
    ax3.set_ylabel('fraction [%]')

    ax4 = fig.add_subplot(gs_left[3,0:2])
    sns.lineplot(data=df_CLR, x='DayOfYear',y='SGE_pw_clr',hue='year',palette=cm.get_sub_cmap('bone', 0.0, 0.7),legend=False)
    ax4.set_ylabel('strength [PW]')

    ax5 = fig.add_subplot(gs_right[0:2,2:4])
    sns.scatterplot(ax=ax5,data=df_ALL,x='SGE_frac_all',y='SGE_pw_all',hue='year',palette='copper')
    #sns.regplot(ax=ax5,data=df_ALL,x='SGE_frac_all',y='SGE_pw_all',scatter=False,color='orange',ci=None,robust=True)
    ax5.set_ylabel(' All-sky \nstrength [TW]')
    ax5.set_xlabel('fraction [%]')

    ax6 = fig.add_subplot(gs_right[2:4,2:4])
    sns.scatterplot(ax=ax6,data=df_CLR,x='SGE_frac_clr',y='SGE_pw_clr',hue='year',palette=cm.get_sub_cmap('bone', 0.0, 0.7))
    #sns.regplot(ax=ax6,data=df_CLR,x='SGE_frac_clr',y='SGE_pw_clr',scatter=False,color='skyblue',ci=None,robust=True)
    ax6.set_ylabel(' Clear-sky \nstrength [PW]')
    ax6.set_xlabel('fraction [%]')

In [ ]:
fig = sns.jointplot(data=df_ALL,x='SGE_frac_all',y='SGE_pw_all',hue='year',palette='copper')
plt.xlabel('fraction [%]',fontsize=20)
plt.ylabel('strength [TW]',fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.legend(fontsize=16)

In [ ]:
fig = sns.jointplot(data=df_CLR,x='SGE_frac_clr',y='SGE_pw_clr',hue='year',palette=cm.get_sub_cmap('bone', 0.0, 0.7))
plt.xlabel('fraction [%]',fontsize=20)
plt.ylabel('strength [PW]',fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.legend(fontsize=16)

In [ ]:
#Figure 3: maps of frequency and strength

#seasonal means
occ_all_seas = SGE_ts.SGE_occ_all.groupby('time.season').mean('time')*100
occ_clr_seas = SGE_ts.SGE_occ_clr.groupby('time.season').mean('time')*100
str_all_seas = SGE_ts.SGE_str_all.groupby('time.season').mean('time')
str_clr_seas = SGE_ts.SGE_str_clr.groupby('time.season').mean('time')

In [ ]:

with sns.plotting_context('talk'):

    fig = plt.figure(figsize=(18,6))
    proj=ccrs.Mollweide()
    all_cmap = 'pink_r'
    clr_cmap = 'ocean_r'
    lats=occ_all_seas.lat
    lons=occ_all_seas.lon
    gs = gridspec.GridSpec(ncols=5, nrows=5, hspace = 0.0, wspace = 0.05, height_ratios=[1,1,1,1,0.43], width_ratios=[1,1,0.2,1,1])

    ax1 = fig.add_subplot(gs[0,0], projection=proj)
    plt.contourf(lons,lats,occ_all_seas.sel(season='DJF'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=5)
    plt.gca().coastlines()

    #plt.subplot(445, projection=proj)
    ax2 = fig.add_subplot(gs[1,0], projection=proj)
    plt.contourf(lons,lats,occ_all_seas.sel(season='MAM'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(449, projection=proj)
    ax3 = fig.add_subplot(gs[2,0], projection=proj)
    plt.contourf(lons,lats,occ_all_seas.sel(season='JJA'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #ax4=plt.subplot(4,4,13, projection=proj)
    ax4 = fig.add_subplot(gs[3,0], projection=proj)
    im=plt.contourf(lons,lats,occ_all_seas.sel(season='SON'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()
    ax5 = fig.add_subplot(gs[4,0], projection=proj)
    plt.colorbar(im,ax=ax5, location='bottom',label='Occurance [%]',fraction=0.75,shrink=0.8)
    fig.delaxes(ax5)


    #plt.subplot(442, projection=proj)
    ax7 = fig.add_subplot(gs[0,1], projection=proj)
    plt.contourf(lons,lats,str_all_seas.sel(season='DJF'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(446, projection=proj)
    ax8 = fig.add_subplot(gs[1,1], projection=proj)
    plt.contourf(lons,lats,str_all_seas.sel(season='MAM'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(4,4,10, projection=proj)
    ax9 = fig.add_subplot(gs[2,1], projection=proj)
    plt.contourf(lons,lats,str_all_seas.sel(season='JJA'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #ax5=plt.subplot(4,4,14, projection=proj)
    ax10 = fig.add_subplot(gs[3,1], projection=proj)
    im2=plt.contourf(lons,lats,str_all_seas.sel(season='SON'),10,transform=ccrs.PlateCarree(),cmap=all_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()
    ax11 = fig.add_subplot(gs[4,1], projection=proj)
    plt.colorbar(im2,ax=ax11, location='bottom',label='Strength [$W/m^2$]',fraction=0.75,shrink=0.8)
    fig.delaxes(ax11)


    #plt.subplot(443, projection=proj)
    ax12 = fig.add_subplot(gs[0,3], projection=proj)
    plt.contourf(lons,lats,occ_clr_seas.sel(season='DJF'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(447, projection=proj)
    ax13 = fig.add_subplot(gs[1,3], projection=proj)
    plt.contourf(lons,lats,occ_clr_seas.sel(season='MAM'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(4,4,11, projection=proj)
    ax13 = fig.add_subplot(gs[2,3], projection=proj)
    plt.contourf(lons,lats,occ_clr_seas.sel(season='JJA'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #ax6=plt.subplot(4,4,15, projection=proj)
    ax14 = fig.add_subplot(gs[3,3], projection=proj)
    im3=plt.contourf(lons,lats,occ_clr_seas.sel(season='SON'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()
    ax15 = fig.add_subplot(gs[4,3], projection=proj)
    plt.colorbar(im3,ax=ax15, location='bottom',label='Occurance [%]',fraction=0.75,shrink=0.8)
    fig.delaxes(ax15)   

    #plt.subplot(444, projection=proj)
    ax16 = fig.add_subplot(gs[0,4], projection=proj)
    plt.contourf(lons,lats,str_clr_seas.sel(season='DJF'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(448, projection=proj)
    ax17 = fig.add_subplot(gs[1,4], projection=proj)
    plt.contourf(lons,lats,str_clr_seas.sel(season='MAM'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #plt.subplot(4,4,12, projection=proj)
    ax18 = fig.add_subplot(gs[2,4], projection=proj)
    plt.contourf(lons,lats,str_clr_seas.sel(season='JJA'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()

    #ax7=plt.subplot(4,4,16, projection=proj)
    ax19 = fig.add_subplot(gs[3,4], projection=proj)
    im4= plt.contourf(lons,lats,str_clr_seas.sel(season='SON'),10,transform=ccrs.PlateCarree(),cmap=clr_cmap)
    plt.gca().add_feature(cartopy.feature.LAND, color='white',zorder=10)
    plt.gca().coastlines()
    ax20 = fig.add_subplot(gs[4,4], projection=proj)
    plt.colorbar(im4,ax=ax20, location='bottom',label='Strength [$W/m^2$]',fraction=0.75,shrink=0.8)
    fig.delaxes(ax20)

fig.tight_layout()
fig.subplots_adjust(top=0.85)  
plt.figtext(0.31,0.88,"All-sky", va="center", ha="center", size=25)
plt.figtext(0.71,0.88,"Clear-sky", va="center", ha="center", size=25)
#add labels into center column:
labels = ['DJF','MAM','JJA','SON']
for ii in range(4):
    ax_label = fig.add_subplot(gs[ii,2])
    ax_label.text(0.5,0.5,labels[ii], horizontalalignment='center', verticalalignment='center', transform=ax_label.transAxes, fontsize=20)
    ax_label.set_axis_off()


old/experimental below

In [ ]:
#Figure 4: maps of grid-wise trends
#remove seasonal cycle and fit linear trend gridwise, then plot slope in each gridpoint.

for ii in range(np.size(SGE_ts.lat)):
    for jj in range(np.size(SGE_ts.lon)):
        stl_occ = STL(SGE_ts.SGE_occ_all[:,ii,jj], period=365, robust=True)



In [ ]:
test_stl=STL(SGE_ts.SGE_str_all[:,20,50], period=365)
test_res = test_stl.fit()
test_res.plot()

#check if STL cannot handle zeros....

In [ ]:
#10pt lowpass butterworth filter:
#copied from Mann 2008: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/2008GL034716, http://www.meteo.psu.edu/holocene/public_html/smoothing08/
#(probably should re-write to proper python package sometime)
'''
def lowpass_mann(indata, f, iconb, icone):
    #f is the cutoff frequency (in cycles/time units)
    #icon is options for boundary constraints:
    # (0) minimum norm,
    # (1) minimum slope,
    # (2) minimum roughness
    padded=()
    ipts=10
    fn=f*2
    npad=3*round(1/fn)
    nn=len(indata)
    padded[npad:npad+nn-1]=indata
    padded[npad+nn:nn+2*npad-1]=indata[nn]
    padded[0:npad-1]=indata[1]
'''

cutoff = 1/30
b, a = signal.butter(10, cutoff, btype='lowpass') 
yfilt = signal.filtfilt(b, a, y, method='pad')
yfilt=np.array(yfilt)
yfilt=yfilt.transpose()

plt.plot(y,'r')
plt.plot(yfilt,'b',linewidth=1)

In [ ]:
#Figure 1: timeseries of SGE frequency and strength for all-sky and clear-sky OLR:

def filter_ts(y,cutoff):
    b, a = signal.butter(10, cutoff, btype='lowpass') 
    yfilt = signal.filtfilt(b, a, y, method='pad')
    yfilt = np.array(yfilt)
    yfilt = yfilt.transpose()
    return yfilt

cf_filt = filter_ts(SGE_timeseres['SGE_frac_clr'], 1/45)
cs_filt = filter_ts(SGE_timeseres['SGE_pw_clr'], 1/45)
af_filt = filter_ts(SGE_timeseres['SGE_frac_all'], 1/45)
as_filt = filter_ts(SGE_timeseres['SGE_pw_all'], 1/45)

X = SGE_timeseres['time'].sortby(SGE_timeseres['time'])

import seaborn as sns
import matplotlib.colors as mcolors

with sns.plotting_context("talk"):

    fig, axes = plt.subplots(2, 2, figsize=(14,12))
    axes[0,0].plot(X, SGE_timeseres['SGE_frac_clr']*100,'c',linewidth=1,alpha=0.5)
    axes[0,0].plot(X, cf_filt*100,'b')

    axes[0,1].plot(X,SGE_timeseres['SGE_pw_clr']/1e12,'c',linewidth=1,alpha=0.5)
    axes[0,1].plot(X, cs_filt/1e12,'b')

    axes[1,0].plot(X, SGE_timeseres['SGE_frac_all']*100,linewidth=1,color='mistyrose')
    axes[1,0].plot(X, af_filt*100,color='sienna')

    axes[1,1].plot(X, SGE_timeseres['SGE_pw_all']/1e14,linewidth=1,color='mistyrose')
    axes[1,1].plot(X, as_filt*100,color='sienna')

In [ ]:
#SGE_timeseries = xr.open_dataset('SGE_timeseries.nc')

#1: deseasonalized timeseries of SGE global area fraction and strength

#plot dfft of fraction, check harmonics:
from scipy.fft import fft, fftfreq

N = len(SGE_frac_clr_clean)
T = 1/365
yf = fft(SGE_frac_clr_clean.values)
xf = fftfreq(N, T)[:N//2]
from scipy.signal import windows
w = windows.hamming(N)
ywf = fft(SGE_frac_clr_clean.values*w)
plt.semilogy(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.semilogy(xf[1:N//2], 2.0/N * np.abs(ywf[1:N//2]), '-r')
plt.grid()
plt.show()
